In [ ]:
from spectralmc.sobol_sampler import SobolSampler, BoundSpec
from spectralmc.gbm import SimulationParams, BlackScholes
import pandas as pd
from typing import Optional

In [ ]:
sp = SimulationParams(
    timesteps=10,
    network_size=256,
    batches_per_mc_run=1024,
    threads_per_block=256,
    mc_seed=42,
    buffer_size=4,
)

In [ ]:
total_size_mb=sp.memory_footprint_bytes()/1024/1024
print(total_size_mb)

In [ ]:
bs_dimensions = {
    "X0": BoundSpec(lower=0.001, upper=10000),
    "K":  BoundSpec(lower=0.001, upper=20000),
    "T":  BoundSpec(lower=0,     upper=10),
    "r":  BoundSpec(lower=-0.2,  upper=0.2),
    "d":  BoundSpec(lower=-0.2,  upper=0.2),
    "v":  BoundSpec(lower=0,     upper=2.0),
}

In [ ]:
ss = SobolSampler(pydantic_class=BlackScholes.Inputs,dimensions=bs_dimensions,seed=43)

In [ ]:
bs = BlackScholes(sp=sp)

In [ ]:
%%time
samples=ss.sample(128)

In [ ]:
%%time
prices=[bs.price_to_host(input) for input in samples]

In [ ]:
bs._normal_gen.get_time_spent_synchronizing()

In [ ]:
from __future__ import annotations

import math
from typing import Final

import QuantLib as ql  # type: ignore[import]

# assume BlackScholes (with Inputs and HostPricingResults) is already defined


def bs_price_quantlib(
    inp: BlackScholes.Inputs,
) -> BlackScholes.HostPricingResults:
    """
    Analytic Black-Scholes–Merton pricer using the one-line
    :pyfunc:`QuantLib.blackFormula`.

    Parameters
    ----------
    inp : BlackScholes.Inputs
        Black-Scholes inputs *(S₀, K, T, r, d, σ)* where ``d`` is the
        continuous dividend or repo yield.

    Returns
    -------
    BlackScholes.HostPricingResults
        ==============  =====================================================
        Field           Definition
        --------------  -----------------------------------------------------
        call_price       QL Black price for a call
        put_price        QL Black price for a put
        call_price_intrinsic   :math:`e^{-rT}\\max(E_Q[S_T]-K,0)`
        put_price_intrinsic    :math:`e^{-rT}\\max(K-E_Q[S_T],0)`
        underlying       Risk-neutral expectation
                         :math:`E_Q[S_T]=S_0e^{(r-d)T}`
        call_convexity   ``0.0``  (placeholder)
        put_convexity    ``0.0``  (placeholder)
        ==============  =====================================================

    Notes
    -----
    * :pyfunc:`QuantLib.blackFormula` signature:
      ``blackFormula(opt_type, strike, forward, std_dev, discount=1.0)``
    * Intrinsic values are discounted *and* based on the **forward**
      expectation rather than spot.
    * Convexity fields are retained for schema compatibility but unused.
    """
    # -------- pre-compute quantities required by blackFormula ------------
    std_dev: Final[float] = inp.v * math.sqrt(inp.T)            # σ√T
    discount: Final[float] = math.exp(-inp.r * inp.T)           # e^{-rT}
    forward: Final[float] = inp.X0 * math.exp((inp.r - inp.d) * inp.T)

    # ---------------- option prices (analytic Black) ---------------------
    call_price: float = ql.blackFormula(
        ql.Option.Call, inp.K, forward, std_dev, discount
    )
    put_price: float = ql.blackFormula(
        ql.Option.Put, inp.K, forward, std_dev, discount
    )

    # ------------- discounted intrinsic values (based on forward) --------
    call_intrinsic: float = discount * max(forward - inp.K, 0.0)
    put_intrinsic: float = discount * max(inp.K - forward, 0.0)

    # --------------------------- assemble result -------------------------
    return BlackScholes.HostPricingResults(
        call_price_intrinsic=call_intrinsic,
        put_price_intrinsic=put_intrinsic,
        underlying=forward,
        put_convexity=call_price-call_intrinsic,
        call_convexity=put_price-put_intrinsic,
        call_price=call_price,
        put_price=put_price,
    )

In [ ]:
%%time
ql_prices=[ bs_price_quantlib (input) for input in samples]

In [ ]:
prices[0]

In [ ]:
ql_prices[0]

In [ ]:
from __future__ import annotations

# --------------------------------------------------------------------------
# Assume the BlackScholes class (with nested pydantic Inputs and
# HostPricingResults) is already in scope.
# --------------------------------------------------------------------------


def black_scholes_pricer(
    inp: BlackScholes.Inputs,
) -> BlackScholes.HostPricingResults:
    r"""
    Closed-form Black-Scholes–Merton valuation.

    Parameters
    ----------
    inp : BlackScholes.Inputs
        ============  =======================================================
        Field         Meaning
        ------------  -------------------------------------------------------
        ``X0``        Spot price :math:`S_0`
        ``K``         Strike
        ``T``         Time to maturity in **years** (ACT/365F)
        ``r``         Continuously-compounded risk-free rate
        ``d``         Continuously-compounded dividend (or repo) yield
        ``v``         Volatility :math:`\sigma`
        ============  =======================================================

    Returns
    -------
    BlackScholes.HostPricingResults
        ================  ===================================================
        Field             Value
        ----------------  ---------------------------------------------------
        ``call_price``    Analytic Black-Scholes call premium
        ``put_price``     Analytic Black-Scholes put  premium
        ``underlying``    Risk-neutral expectation
                          :math:`E_Q[S_T]=S_0 e^{(r-d)T}`
        ``call_price_intrinsic``  Discounted intrinsic call value  
                          :math:`e^{-rT}\max(E_Q[S_T]-K,0)`
        ``put_price_intrinsic``   Discounted intrinsic put  value  
                          :math:`e^{-rT}\max(K-E_Q[S_T],0)`
        ``call_convexity``        *Time value* of the call  
                          (= price − intrinsic)
        ``put_convexity``         *Time value* of the put
        ================  ===================================================

    Notes
    -----
    * The cumulative normal distribution is implemented via
      ``math.erf`` to avoid extra dependencies.
    * Convexity (sometimes called *extrinsic* or *time value*) is defined
      here as **option price minus discounted intrinsic value**.
    """
    # -------- standard normal CDF ---------------------------------------
    def _norm_cdf(x: float) -> float:
        return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

    # -------- derived quantities ----------------------------------------
    sqrt_T = math.sqrt(inp.T) if inp.T > 0.0 else 1e-16
    d1 = (
        math.log(inp.X0 / inp.K)
        + (inp.r - inp.d + 0.5 * inp.v * inp.v) * inp.T
    ) / (inp.v * sqrt_T)
    d2 = d1 - inp.v * sqrt_T

    discount = math.exp(-inp.r * inp.T)              # e^{-rT}
    dividend = math.exp(-inp.d * inp.T)              # e^{-dT}
    forward  = inp.X0 * math.exp((inp.r - inp.d) * inp.T)  # E_Q[S_T]

    # -------- Black-Scholes prices --------------------------------------
    call_price = inp.X0 * dividend * _norm_cdf(d1) - inp.K * discount * _norm_cdf(d2)
    put_price  = inp.K * discount * _norm_cdf(-d2) - inp.X0 * dividend * _norm_cdf(-d1)

    # -------- discounted intrinsic values (based on forward) ------------
    call_intrinsic = discount * max(forward - inp.K, 0.0)
    put_intrinsic  = discount * max(inp.K - forward, 0.0)

    # -------- convexities (time value) ----------------------------------
    call_convexity = call_price - call_intrinsic
    put_convexity  = put_price  - put_intrinsic

    # -------- assemble result ------------------------------------------
    return BlackScholes.HostPricingResults(
        call_price_intrinsic=call_intrinsic,
        put_price_intrinsic=put_intrinsic,
        underlying=forward,
        put_convexity=put_convexity,
        call_convexity=call_convexity,
        call_price=call_price,
        put_price=put_price,
    )

In [ ]:
inputs=BlackScholes.Inputs(
    X0=100,
    K=100,
    T=2,
    r=0.02,
    d=0.05,
    v=0.2,
)

In [ ]:
bs.price_to_host(inputs)

In [ ]:
bs_price_quantlib(inputs)

In [ ]:
black_scholes_pricer(inputs)